In [ ]:
####### Run a simple analysis of terrain around a selected location

# Step 1: Sign up for a Google Earth Engine Account here:
# https://earthengine.google.com/signup/

# you will use this to authenticate your account with GEE to run the script and
# utilize GEE layers

In [ ]:
# get any missing modules
! pip install earthengine-api geemap

In [1]:
# setup modules
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import sys,os,os.path,tempfile

# spatial modules
import ee
#import geemap # currently not using as colorbars dont load properly in google collab
import geemap.colormaps as cm
import geemap.foliumap as geemap # use folium instead so coloramps work on goolge collab

In [2]:
## setup output. defined dir or system temp dir
data_dir=os.path.expanduser(os.path.join(tempfile.gettempdir(),'terrain_analysis_output'))

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(" ")
print("****** Raster and blockage results output folder ******")
print("****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******")
print("****** Use the dir navigation to go up one level, then navigate to tmp/geo ******")
print(data_dir)

 
****** Raster and blockage results output folder ******
****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******
****** Use the dir navigation to go up one level, then navigate to tmp/geo ******
/var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output


In [3]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the authentication flow. You only need to do this once while running notebook
#ee.Authenticate()

## After inserting the API key initialize GEE
ee.Initialize()

In [5]:
bbox = ee.Geometry.Rectangle(-94.614744,29.317645,-74.441221,36.520919)

nexrad = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_locations').filterBounds(bbox)
AandM_EC_Tower = ee.FeatureCollection('users/serbinsh/vector_data/amf3/BNF_AandM_EC_Tower')
potential_BNF_AOS_sites = ee.FeatureCollection('users/serbinsh/vector_data/amf3/potential_BNF_AOS_sites')
AandM_EC_Tower_2kmbuff = ee.FeatureCollection('users/serbinsh/vector_data/amf3/BNF_AandM_EC_Tower_2kmbuff_AlbersEA')
nexrad = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_locations').filterBounds(bbox)
nexrad_100km = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_100km_buffer_AEA').filterBounds(bbox)

In [6]:
# DEM
elev_dataset = ee.Image('USGS/NED')
ned_elevation = elev_dataset.select('elevation')
dem_palette = cm.get_palette('Spectral_r', n_class=30)
demViz = {'min': 0.0, 'max': 2000.0, 'palette': dem_palette, 'opacity': 1}

In [7]:
#create hillshade & slope
ned_hillshade = ee.Terrain.hillshade(ned_elevation)
ned_slope = ee.Terrain.slope(ned_elevation)


In [17]:
# // ********* Display the data! ************* //

srtm = geemap.Map(center=[33.315809,-85.198609], zoom=7)
srtm.addLayer(ned_elevation, demViz, 'Elevation above sea level');
srtm.addLayer(AandM_EC_Tower, {'color': 'black', 'strokeWidth': 1}, 'A & M Tower')

srtm.addLayer(nexrad,{'color': 'yellow', 'strokeWidth': 3},'NEXRAD locations')
srtm.addLayer(nexrad_100km, {'color': '#d3d3d3', 'opacity': 0.20}, 'NEXRAD 100km buffers')

srtm.add_colorbar(colors=demViz['palette'], vmin=demViz['min'], 
                  vmax=demViz['max'], label="Elevation (m)", 
                  layer_name="SRTM DEM", position="bottomright")
states = ee.FeatureCollection('TIGER/2018/States')
statesImage = ee.Image().paint(states, 0, 2)
srtm.addLayer(statesImage, {'palette': 'black'}, 'US States')
srtm.addLayerControl()
srtm